In [8]:
import pandas as pd

df_all = pd.concat([
    pd.read_csv('input/candle.{}.csv'.format(i)).assign(instrument=i, time=lambda d: pd.to_datetime(d['time']))
    for i in ['EUR_USD', 'USD_JPY', 'GBP_USD', 'AUD_USD']
]).set_index(['instrument', 'time'])
df_all

openBid  openAsk  highBid  highAsk   lowBid  \
instrument time                                                               
EUR_USD    2018-10-12 13:02:55  1.15552  1.15564  1.15556  1.15569  1.15551   
           2018-10-12 13:03:00  1.15558  1.15571  1.15559  1.15572  1.15558   
           2018-10-12 13:03:05  1.15558  1.15573  1.15558  1.15573  1.15551   
           2018-10-12 13:03:10  1.15550  1.15563  1.15550  1.15563  1.15543   
           2018-10-12 13:03:15  1.15546  1.15559  1.15548  1.15561  1.15543   
           2018-10-12 13:03:20  1.15549  1.15560  1.15549  1.15561  1.15544   
           2018-10-12 13:03:25  1.15544  1.15557  1.15548  1.15561  1.15544   
           2018-10-12 13:03:30  1.15548  1.15560  1.15558  1.15572  1.15546   
           2018-10-12 13:03:35  1.15560  1.15571  1.15576  1.15590  1.15558   
           2018-10-12 13:03:40  1.15577  1.15590  1.15585  1.15599  1.15577   
           2018-10-12 13:03:45  1.15582  1.15595  1.15582  1.15595  1.15576   
           2018-10-12 13:03:50  1.15582  1.15595  1.15586  1.15599  1.15582   
           2018-10-12 13:03:55  1.15586  1.15598  1.15596  1.15609  1.15581   
           2018-10-12 13:04:00  1.15584  1.15596  1.15590  1.15603  1.15578   
           2018-10-12 13:04:05  1.15584  1.15596  1.15592  1.15605  1.15583   
           2018-10-12 13:04:10  1.15584  1.15596  1.15584  1.15596  1.15576   
           2018-10-12 13:04:15  1.15580  1.15593  1.15581  1.15595  1.15578   
           2018-10-12 13:04:20  1.15584  1.15596  1.15592  1.15605  1.15584   
           2018-10-12 13:04:25  1.15590  1.15603  1.15590  1.15603  1.15581   
           2018-10-12 13:04:30  1.15582  1.15594  1.15586  1.15599  1.15574   
           2018-10-12 13:04:35  1.15578  1.15592  1.15582  1.15594  1.15578   
           2018-10-12 13:04:40  1.15581  1.15593  1.15581  1.15593  1.15579   
           2018-10-12 13:04:45  1.15578  1.15590  1.15582  1.15595  1.15575   
           2018-10-12 13:04:50  1.15579  1.15592  1.15582  1.15596  1.15575   
           2018-10-12 13:04:55  1.15576  1.15589  1.15585  1.15599  1.15576   
           2018-10-12 13:05:00  1.15585  1.15598  1.15591  1.15604  1.15584   
           2018-10-12 13:05:05  1.15588  1.15601  1.15592  1.15604  1.15583   
           2018-10-12 13:05:15  1.15582  1.15595  1.15583  1.15596  1.15581   
           2018-10-12 13:05:20  1.15582  1.15595  1.15592  1.15604  1.15582   
           2018-10-12 13:05:25  1.15589  1.15603  1.15589  1.15603  1.15582   
...                                 ...      ...      ...      ...      ...   
AUD_USD    2018-10-12 20:55:20  0.71145  0.71171  0.71146  0.71171  0.71144   
           2018-10-12 20:55:25  0.71145  0.71170  0.71145  0.71170  0.71145   
           2018-10-12 20:55:35  0.71141  0.71169  0.71141  0.71169  0.71141   
           2018-10-12 20:55:45  0.71145  0.71173  0.71147  0.71175  0.71145   
           2018-10-12 20:55:50  0.71149  0.71175  0.71149  0.71176  0.71147   
           2018-10-12 20:56:00  0.71146  0.71176  0.71158  0.71181  0.71145   
           2018-10-12 20:56:05  0.71157  0.71179  0.71157  0.71179  0.71157   
           2018-10-12 20:57:00  0.71144  0.71173  0.71150  0.71173  0.71142   
           2018-10-12 20:57:05  0.71149  0.71172  0.71149  0.71172  0.71149   
           2018-10-12 20:57:15  0.71149  0.71171  0.71149  0.71171  0.71149   
           2018-10-12 20:57:20  0.71148  0.71172  0.71148  0.71172  0.71148   
           2018-10-12 20:57:40  0.71147  0.71176  0.71150  0.71176  0.71147   
           2018-10-12 20:57:45  0.71146  0.71176  0.71150  0.71176  0.71146   
           2018-10-12 20:57:50  0.71150  0.71174  0.71150  0.71174  0.71150   
           2018-10-12 20:57:55  0.71147  0.71176  0.71147  0.71176  0.71147   
           2018-10-12 20:58:00  0.71149  0.71175  0.71150  0.71175  0.71149   
           2018-10-12 20:58:05  0.71149  0.71175  0.71150  0.71175  0.71149   
           2018-10-12 20:58:10  0.71149  0.71176  0.71150  0.71176  0.71149   
   